In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"

from torch.utils.data import DataLoader

from preprocessing.get_en_dataloader import get_training_dev_test_dataset

In [2]:
train_dataset, dev_dataset, test_dataset = get_training_dev_test_dataset(max_length=128)

209899
16496
31064


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from utils.sim_bert import CustomBertForSequenceSimilarity

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    fp16=True,
)

model = CustomBertForSequenceSimilarity.from_pretrained("bert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=dev_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceSimilarity: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing CustomBertForSequenceSimilarity from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceSimilarity from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceSimilarity were not initialized from the model check

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/home/napat-nii/.pyenv/versions/3.7.7/envs/nlu/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/napat-nii/.pyenv/versions/3.7.7/envs/nlu/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss
10,0.742132
20,0.599769
30,0.436072
40,0.340521
50,0.330147
60,0.326890
70,0.326323
80,0.325629
90,0.325267
100,0.325425


In [8]:
def compute_metrics(p):
    preds = p.logits
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == 0).astype(np.float32).mean().item()}

In [9]:
trainer.compute_metrics = compute_metrics

In [10]:
trainer.evaluate()

/home/napat-nii/.pyenv/versions/3.7.7/envs/nlu/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'epoch': 3.0}

In [11]:
preds = trainer.predict(dev_dataset)

/home/napat-nii/.pyenv/versions/3.7.7/envs/nlu/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [12]:
preds

PredictionOutput(predictions=(array([   2.3025937,    2.3026052,    2.3026133, ..., -100.       ,
       -100.       , -100.       ], dtype=float32), array([[-0.14575195, -0.14562988, -0.14562988, ..., -0.14562988,
        -0.14575195, -0.14575195],
       [-0.14538574, -0.14538574, -0.14538574, ..., -0.14538574,
        -0.14538574, -0.14538574],
       [-0.14550781, -0.14550781, -0.14550781, ..., -0.14538574,
        -0.14562988, -0.14550781],
       ...,
       [-0.14562988, -0.14550781, -0.14538574, ..., -0.14550781,
        -0.14550781, -0.14550781],
       [-0.14575195, -0.14562988, -0.14575195, ..., -0.14575195,
        -0.14575195, -0.14575195],
       [-0.14562988, -0.14562988, -0.14562988, ..., -0.14575195,
        -0.14575195, -0.14562988]], dtype=float32)), label_ids=None, metrics={})

In [15]:
preds.predictions[0][:100]

array([2.3025937, 2.3026052, 2.3026133, 2.3026233, 2.3026257, 2.3025951,
       2.3026333, 2.3025796, 2.3025835, 2.3025765, 2.3026216, 2.302579 ,
       2.302591 , 2.3025775, 2.3025866, 2.3026009, 2.302617 , 2.3026059,
       2.3025677, 2.3026087, 2.3026705, 2.3026164, 2.302581 , 2.3026083,
       2.3025665, 2.302591 , 2.3025966, 2.3025765, 2.3025577, 2.3025856,
       2.3026004, 2.3025997, 2.3025963, 2.302576 , 2.302561 , 2.3025904,
       2.3025918, 2.302591 , 2.302568 , 2.3025908, 2.302588 , 2.3025727,
       2.3026118, 2.3025935, 2.302592 , 2.3025737, 2.302583 , 2.3026032,
       2.3026102, 2.3025887, 2.302589 , 2.302564 , 2.3025942, 2.3026097,
       2.3025713, 2.3025827, 2.302582 , 2.3026187, 2.302604 , 2.302608 ,
       2.3025959, 2.3025944, 2.3025966, 2.302606 , 2.3025985, 2.3026073,
       2.3025904, 2.3025882, 2.3025837, 2.3025723, 2.302608 , 2.3026102,
       2.3026087, 2.3025723, 2.3026016, 2.302607 , 2.3025968, 2.3026016,
       2.302628 , 2.3025796, 2.3025784, 2.3025994, 

In [ ]:
1

In [ ]:
2